In [1]:
# import pandas as pd
# import sqlalchemy
# #from sqlalchemy import create_engine
# import pyhive
# from sqlalchemy.engine import create_engine

In [2]:
# Presto
# engine = create_engine('presto://localhost:8080/system/runtime') 

In [3]:
#Read Presto Data query into a DataFrame
# df = pd.read_sql('select count(*) from sword_purchases', engine)
# df.head()

In [4]:
# from pyhive import presto
# cursor = presto.connect(host='localhost', port=8080).cursor()
# statement = 'select count(*) from sword_purchases'
# cursor.execute(statement)
# my_results = cursor.fetchall()

### Summary Instructions:

**(1). Activate Docker Compose Cluster:**
```
docker-compose up -d
```

**(2). Run Game Application:**
```
docker-compose exec mids env FLASK_APP=/w205/project-3-cmorenoUCB2021/game_api.py flask run --host 0.0.0.0
```

**(3). Set up to Watch Kafka:** to observe how messages are being captured. Open a new terminal, and run the following (run twice):
```
docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning
```

**(4). Run Application to Read Messages from Kafka and write them to hdfs:** using separate terminals, run applications to read Messages from Kafka associated with the key events from the game as follows:  

```
docker-compose exec spark spark-submit /w205/project-3-cmorenoUCB2021/write_events_stream.py
```
  
**(4). Check what was written in Hadoop:** the following command would check what was written for sword_purchases. 
Note: check files for each event (for sanity check).
```
docker-compose exec cloudera hadoop fs -ls /tmp/purchase_events
```
```
docker-compose exec cloudera hadoop fs -ls /tmp/guild_events
```

**(5). Set up Presto:**

*(a). Run hive in hadoop container:*
```
docker-compose exec cloudera hive
```
*(b). Create tables:*
**purchase_events**
```
create external table if not exists default.purchase_events (
    raw_event string,
    timestamp string,
    Accept string,
    Host string,
    User_Agent string,
    event_type string,
    price string,
    n_purchased string,
    strength string,
    name string,
    userid string
  )
  stored as parquet 
  location '/tmp/purchase_events'
  tblproperties ("parquet.compress"="SNAPPY");
```
```
create external table if not exists default.purchase_events (price string, n_purchased string, strength string, name string, userid string, Accept string, Host string, User_Agent string, event_type string, timestamp string, raw_event string) stored as parquet location '/tmp/purchase_events' tblproperties ("parquet.compress"="SNAPPY");

create external table if not exists default.purchase_events (event_ts string, userid string, Host string, event_type string, name string, strength string, n_purchased int, price string) stored as parquet location '/tmp/purchase_events' tblproperties ("parquet.compress"="SNAPPY");

```

**guild_events**  
```
create external table if not exists default.guild_events (
    raw_event string,
    timestamp string,
    Accept string,
    Host string,
    User_Agent string,
    event_type string,
    price string,
    n_purchased string,
    strength string,
    name string,
    userid string
  )
  stored as parquet 
  location '/tmp/guild_events'
  tblproperties ("parquet.compress"="SNAPPY");
```
```
create external table if not exists default.guild_events (price int, n_purchased int, strength int, name string, userid string, Accept string, Host string, User_Agent string, event_type string, timestamp string, raw_event string) stored as parquet location '/tmp/guild_events' tblproperties ("parquet.compress"="SNAPPY");

create external table if not exists default.guild_events (event_ts string, userid string, Host string, event_type string, name string, strength string, n_purchased int, price string) stored as parquet location '/tmp/guild_events' tblproperties ("parquet.compress"="SNAPPY");
```

**Note:** `ctrl-D` to exit the hive shell.

**(6). Query Tables with Presto:**  

&nbsp;&nbsp;&nbsp;**(a). Run Presto:**
```
docker-compose exec presto presto --server presto:8080 --catalog hive --schema default
```
&nbsp;&nbsp;&nbsp;**(b). Examples of Queries with Presto:**

&nbsp;&nbsp;&nbsp;&nbsp;-What tables are there in Presto?
```
presto:default> show tables;
```
&nbsp;&nbsp;&nbsp;&nbsp;-Describe one of the tables (sword_purchases):
```
presto:default> describe purchase_events;
presto:default> describe guild_events;

```

&nbsp;&nbsp;&nbsp;&nbsp;-Query `purchases` table:  
```
presto:default> select * from purchase_events;
presto:default> select * from guild_events;
```
&nbsp;&nbsp;&nbsp;&nbsp;-Count the number of events in `purchases` table:  
```
presto:default> select count(*) from purchase_events;
presto:default> select count(*) from guild_events;
```

**(7). Use Apache Bench to generate data:**  

u = Number of Users
e = Number of endpoints (purchase_a_sword, purchase_a_knife, purchase_a_shield, join_guild)
n = Number of total requests
```
bash data_generator.sh -u 10 -e 4 -n 10 -b
bash data_generator.sh -u 10 -e 4 -n 10
```

```
docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/
docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_sword
docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/
docker-compose exec mids ab -n 10 -H "Host: user2.att.com" http://localhost:5000/purchase_a_sword
```
  
*Another way to feed data to the stream:*  
```
while true; do
  docker-compose exec mids \
    ab -n 10 -H "Host: user1.comcast.com" \
      http://localhost:5000/purchase_a_sword
  sleep 10
done
```
```
while true; do docker-compose exec mids ab -n 10 -H "Host: user1.comcast.com" http://localhost:5000/purchase_a_sword; sleep 10; done
```

**(8). HOW TO RUN PRESTO IN JUPYTER NOTEBOOK:?**
- ** Run spark to make sure it is available when running Jupyter Notebook:**

```
docker-compose exec spark ln -s /w205 w205
```

- **Run Jupyter Notebook in Google Cloud to read kafka topic and explore data:**
```
docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 7000 --ip 0.0.0.0 --allow-root' pyspark
```

- **Get the token and include the address for your notebook instance in Google Cloud:** For example:
```
http://34.139.108.62:7000/?token=9d42832da7a128cbc081a3eb4a5f30b70c853987d8666395

http://34.139.108.62:7000/?token=5e8d32cf031f952660bd997334d969cfdb2bdf1b9c5412f6

http://34.139.108.62:7000/?token=8659ee3af97d274faa78cd16ee3ae349590604a0f1d2d669
```
Replace 0.0.0.0 with the address associated to your Google Cloud Instance.

34.139.108.62

## APPENDIX

### I. Docker Compose Content

```
version: '2'
services:
  zookeeper:
    image: confluentinc/cp-zookeeper:latest
    environment:
      ZOOKEEPER_CLIENT_PORT: 32181
      ZOOKEEPER_TICK_TIME: 2000
    expose:
      - "2181"
      - "2888"
      - "32181"
      - "3888"
    extra_hosts:
      - "moby:127.0.0.1"

  kafka:
    image: confluentinc/cp-kafka:latest
    depends_on:
      - zookeeper
    environment:
      KAFKA_BROKER_ID: 1
      KAFKA_ZOOKEEPER_CONNECT: zookeeper:32181
      KAFKA_ADVERTISED_LISTENERS: PLAINTEXT://kafka:29092
      KAFKA_OFFSETS_TOPIC_REPLICATION_FACTOR: 1
    expose:
      - "9092"
      - "29092"
    extra_hosts:
      - "moby:127.0.0.1"

  cloudera:
    image: midsw205/hadoop:0.0.2
    hostname: cloudera
    expose:
      - "8020" # nn
      - "8888" # hue
      - "9083" # hive thrift
      - "10000" # hive jdbc
      - "50070" # nn http
    ports:
      - "8888:8888"
    extra_hosts:
      - "moby:127.0.0.1"

  spark:
    image: midsw205/spark-python:0.0.6
    stdin_open: true
    tty: true
    volumes:
      - ~/w205:/w205
    expose:
      - "8888"
    ports:
      - "8889:8888" # 8888 conflicts with hue
    depends_on:
      - cloudera
    environment:
      HADOOP_NAMENODE: cloudera
      HIVE_THRIFTSERVER: cloudera:9083
    extra_hosts:
      - "moby:127.0.0.1"
    command: bash

  presto:
    image: midsw205/presto:0.0.1
    hostname: presto
    volumes:
      - ~/w205:/w205
    expose:
      - "8080"
    environment:
      HIVE_THRIFTSERVER: cloudera:9083
    extra_hosts:
      - "moby:127.0.0.1"

  mids:
    image: midsw205/base:0.1.9
    stdin_open: true
    tty: true
    volumes:
      - ~/w205:/w205
    expose:
      - "5000"
    ports:
      - "5000:5000"
    extra_hosts:
      - "moby:127.0.0.1"
```

### II. Game Application - Python API

```
#!/usr/bin/env python
import json
from kafka import KafkaProducer
from flask import Flask, request

app = Flask(__name__)
producer = KafkaProducer(bootstrap_servers='kafka:29092')


def log_to_kafka(topic, event):
    event.update(request.headers)
    producer.send(topic, json.dumps(event).encode())


@app.route("/")
def default_response():
    default_event = {'event_type': 'default',
                     'name': 'doing_nothing',
                     'strength':'NA',
                     'price': 'NA'}
    log_to_kafka('events', default_event)
    return "What are you waiting for?\n"


@app.route("/purchase_a_sword", methods=['POST','GET'])
    """
    @function: This function generate a Purchase a Sword event from a user mobile device request or Apache Bench
    @param: User Request (via URL endpoint) 
    @return: Returns string of User Id and Event 
    """
def purchase_a_sword():
    userid = request.args.get('userid', default='001', type=str)
    n = request.args.get('n',default=1,type=int)
    purchase_sword_event = {'userid':userid,
                            'event_type': 'purchase_sword',
                            'name': 'excalibur',
                            'strength': '1000',
                            'n_purchases': n,
                            'price': 2000}
    log_to_kafka('events', purchase_sword_event)
    return "USER " + userid + ": "+ str(n)+" "+ " Sword(s) Purchased!\n"


@app.route("/join_guild/", methods=['POST','GET'])
def join_guild():
    """
    @function: This function generate a Join Guild event from a user mobile device request or Apache Bench
    @param: User Request (via URL endpoint) 
    @return: Returns string of User Id and Event 
    """
    userid = request.args.get('userid', default='001', type=str)
    guild_name = request.args.get('guild_name',default="Knights of the Round Table",type=str)
    join_guild_event = {'userid': userid,
                        'event_type': 'join_guild',
                        'name': guild_name,
                        'strength': 1200,
                        'n_purchases': '1',
                        'price': 1000}
    log_to_kafka('events', join_guild_event)
    return "Joined" +" "+ guild_name +" "+ "Guild!\n"


@app.route("/purchase_a_knife")
def purchase_a_knife():
    userid = request.args.get('userid', default='001', type=str)
    n = request.args.get('n',default=1,type=int)
    purchase_knife_event = {'userid': userid,
                            'event_type': 'purchase_knife',
                            'name': 'kukri',
                            'strength': 500,
                            'n_purchases': n,
                            'price': 1000}
    log_to_kafka('events', purchase_knife_event)
    return "USER " + userid + ": "+ str(n)+" "+ " Kniefe(s) Purchased!\n"

@app.route("/purchase_a_shield")
def purchase_a_shield():
    userid = request.args.get('userid', default='001', type=str)
    n = request.args.get('n',default=1,type=int)
    purchase_shield_event = {'userid': useride,
                             'event_type': 'purchase_shield',
                             'name': 'parma',
                             'strength': 800,
                             'n_purchases': n,
                             'price': 1500}
    log_to_kafka('events', purchase_shield_event)
    return "USER " + userid + ": "+ str(n)+" "+ " Shield(s) Purchased!\n"
```

### III. Applications to Extract events from kafka and write them to hdfs

#### Application to Read and Write Event Stream

```
#!/usr/bin/env python
"""Extract events from kafka and write them to hdfs
"""
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, from_json
from pyspark.sql.types import StructType, StructField, StringType


def purchase_sword_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
    ])


@udf('boolean')
def is_sword_purchase(event_as_json):
    """udf for filtering events
    """
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_sword':
        return True
    return False


def main():
    """main
    """
    spark = SparkSession \
        .builder \
        .appName("ExtractEventsJob") \
        .getOrCreate()

    raw_events = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "events") \
        .load()

    sword_purchases = raw_events \
        .filter(is_sword_purchase(raw_events.value.cast('string'))) \
        .select(raw_events.value.cast('string').alias('raw_event'),
                raw_events.timestamp.cast('string'),
                from_json(raw_events.value.cast('string'),
                          purchase_sword_event_schema()).alias('json')) \
        .select('raw_event', 'timestamp', 'json.*')

    sink = sword_purchases \
        .writeStream \
        .format("parquet") \
        .option("checkpointLocation", "/tmp/checkpoints_for_sword_purchases") \
        .option("path", "/tmp/sword_purchases") \
        .trigger(processingTime="120 seconds") \
        .start()

    sink.awaitTermination()


if __name__ == "__main__":
    main()
```

#### Useful References
- https://towardsdatascience.com/jupyter-magics-with-sql-921370099589

**Imports for working with Presto:**
pip install pandas
pip install sqlalchemy # ORM for databases
pip install ipython-sql # SQL magic function